### imports

In [1]:
import torch
torch.cuda.empty_cache()

!pip install -qU langchain tiktoken langchain_community langchain_chroma langchain-huggingface huggingface-hub sentence_transformers chromadb langchainhub transformers peft
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.0/603.0 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 86.6 MB/s eta 0:00:

In [2]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip install markdown --quiet
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 24.3 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 45.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 74.6 MB/s eta 

In [3]:
from langchain_huggingface import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig
from IPython.display import display, Markdown
import gradio as gr
from google.colab import drive
import os
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
from huggingface_hub import login

# Replace 'your_access_token_here' with your actual access token
login(token=#insertyourkey)

# Verify login
from huggingface_hub import HfApi

api = HfApi()

try:
    user_info = api.whoami()
    print("Successfully authenticated! User info:", user_info)
except Exception as e:
    print("Authentication error:", e)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Successfully authenticated! User info: {'type': 'user', 'id': '66c85ffe7d9956c70e4e6b5d', 'name': 'ayshwaryaninet1', 'fullname': 'Shyam', 'email': 'ayshwaryaninet1@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/0591821f79c79ff8d50e97d4ac449afc.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'fine_tuning', 'role': 'write', 'createdAt': '2024-10-13T11:42:16.260Z'}}}


### Import LLM

In [5]:
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "unsloth/phi-3.5-mini-instruct-bnb-4bit"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    attn_implementation="flash_attention_2",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

config = PeftConfig.from_pretrained("ayshwaryaninet1/lora_model19")
model = PeftModel.from_pretrained(model, "ayshwaryaninet1/lora_model19")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Move the model to the correct device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

### Use LLM

In [6]:
import torch
torch.cuda.empty_cache()
!pip install -U bitsandbytes
!pip install transformers datasets accelerate peft datasets
!pip install -qU langchain tiktoken langchain_community langchain_chroma langchain-huggingface huggingface-hub sentence_transformers chromadb langchainhub transformers peft
!pip install flash-attn --no-build-isolation

# Import the necessary modules
!pip install -U langchain
from langchain.embeddings import HuggingFaceBgeEmbeddings # This import is missing
from langchain.vectorstores import Chroma
from IPython.display import display, Markdown

In [20]:
# Initialize embeddings
embedding_model_name = "BAAI/bge-small-en-v1.5"
embedding_model_kwargs = {"device": "cuda"}
embedding_encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=embedding_model_kwargs,
    encode_kwargs=embedding_encode_kwargs
)

# Initialize vector store and retriever
vectorstore = Chroma(
    persist_directory="/content/drive/MyDrive/RAG/vector1",
    embedding_function=hf
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Function to calculate the total number of tokens in the vector database
def count_total_tokens_in_vectorstore(vectorstore, tokenizer):
    # Retrieve all documents from the vector store
    all_docs = vectorstore.get()['documents']

    total_tokens = 0

    # Iterate over each document and calculate the number of tokens
    for doc in all_docs:
        tokens_in_doc = len(tokenizer.encode(doc))  # Tokenize the document content (which is a string)
        total_tokens += tokens_in_doc

    return total_tokens


class RAGChatModel:
    def __init__(self, retriever, model, tokenizer, device, max_token_limit):
        self.retriever = retriever
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.max_token_limit = max_token_limit
        self.current_token_count = 0

        # Templates for prompt generation
        self.template_standard = """
        <|system|>
        Answer the question based on the information provided in the context.
        Providing the content reference page numbers is critical for the answer.

        Context: {context}

        <|user|>
        Question: {question}
        <|end|>

        <|assistant|>
        """

        self.template_exceeded = """
        <|system|>
        Answer the question in detail. However, due to the token limit,
        this response does not contain the full content of the prescribed textbook.
        Still, **you must reference the relevant page numbers** from the retrieved content where possible.

        Context: {context}
        <|user|>
        Question: {question}
        <|end|>

        <|assistant|>
        """

    def num_tokens_from_string(self, string: str) -> int:
        """Returns the number of tokens in a text string using the tokenizer."""
        return len(self.tokenizer.encode(string))

    def format_docs(self, docs, full_content=True, max_pages=3):
        """Format the documents to be used as context in the prompt, limiting to max_pages."""
        if full_content:
            # Include only the first 'max_pages' documents in the response
            formatted_docs = "\n\n".join(f"Information in Page number: {(doc.metadata['page']+1)}\n{doc.page_content}" for doc in docs[:max_pages])
        else:
            # List only the top 'max_pages' page numbers
            formatted_docs = "Information available in prescribed textbook " + ", ".join(f"Page number: {doc.metadata['page']}" for doc in docs[:max_pages])
        return formatted_docs

    def get_prompt(self, docs, question):
        """Generate the prompt based on token count and context formatting."""
        # Format the retrieved documents to be used as context
        context = self.format_docs(docs, full_content=True, max_pages=3)  # Limit pages to 3
        total_tokens_in_context = self.num_tokens_from_string(context)

        # Update the token count
        self.current_token_count += total_tokens_in_context

        # Determine the appropriate template based on token count
        if self.current_token_count > self.max_token_limit:
            print("Token limit exceeded. Information from prescribed textbook will not be used.")
            context = self.format_docs(docs, full_content=False, max_pages=3)
            template = self.template_exceeded
        else:
            template = self.template_standard

        # Format the prompt with the context and question
        prompt = template.format(context=context, question=question)
        return prompt

    def ask_question(self, question):
        """Main function to retrieve relevant docs and generate a response."""
        # Retrieve relevant documents
        docs = self.retriever.invoke(question)

        # Generate the prompt
        prompt = self.get_prompt(docs, question)

        # Tokenize the prompt and move input tensors to the correct device
        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True).to(self.device)

        # Ensure the model is on the same device as the inputs
        self.model = self.model.to(self.device)

        # Generate the response
        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,  # Include the attention mask to handle padding
                max_new_tokens=50,  # Set max tokens to a value > 0, like 50
                num_beams=5,        # Increase number of beams for better response quality
                early_stopping=True, # Stop generation once the model completes a coherent response
                temperature=0.7,    # Control the randomness of responses
                top_p=0.9,          # Use nucleus sampling to add diversity to the response
                repetition_penalty=1.2  # Penalize repetitive sequences
            )

        # Decode the generated output
        raw_answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract and display the clean answer
        clean_answer = self.extract_clean_answer(raw_answer)
        print("Answer:", clean_answer)

    def extract_clean_answer(self, raw_output):
        """Extract only the answer from the raw output."""
        assistant_tag = "<|assistant|>"
        if assistant_tag in raw_output:
            clean_answer = raw_output.split(assistant_tag)[-1].strip()
            return clean_answer
        return raw_output.strip()


In [21]:
# Check the available device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# Calculate max_token_limit for the model
max_token_limit = count_total_tokens_in_vectorstore(vectorstore, tokenizer) // 10

# Initialize the RAGChatModel class and pass the device and max_token_limit
rag_chat_model = RAGChatModel(
    retriever=retriever,    # Your document retriever
    model=model,            # Fine-tuned model loaded with LoRA
    tokenizer=tokenizer,    # Tokenizer corresponding to the model
    device=device,          # Pass the device ('cuda' or 'cpu')
    max_token_limit=max_token_limit  # Set the max token limit
)


In [23]:
import gradio as gr

# Function to shorten the question for the chat history display
def get_short_overview(question, answer, max_length=110000):
    """Generate a short summary of the question for the chat history."""
    short_question = (question[:max_length] + '...') if len(question) > max_length else question
    return short_question

# Function for the RAG model interaction
def ask_question_gradio(history, question):
    if not question:
        return history, ""  # Return empty if no question is asked

    # Call the ask_question method in RAGChatModel
    rag_chat_model.ask_question(question)  # This method should output directly to the interface
    answer = rag_chat_model.extract_clean_answer(rag_chat_model.ask_question(question))  # Get the answer from the RAGChatModel

    # Add the question and answer to the conversation history in the correct format
    history.append({"role": "user", "content": question})
    history.append({"role": "assistant", "content": answer})
    chat_history = "\n\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in history])

    return history, chat_history  # Return the updated history

# Create Gradio Blocks interface
with gr.Blocks() as demo:
    gr.Markdown(
        """
        <h1 style='text-align: center;'>Unsloth ChemNerd Chat</h1>
        <p style='text-align: center;'>Ask any question and get a response from the RAG model.</p>
        """
    )

    with gr.Row():
        with gr.Column(scale=1, min_width=200):
            gr.Markdown("### Chat History Overview")
            history_display = gr.Textbox(label="Chat History", lines=20, interactive=False)

        with gr.Column(scale=2):
            chatbot = gr.Chatbot(label="Chat with RAG Model", type='messages')  # Specify type as 'messages'
            user_input = gr.Textbox(placeholder="Ask your question...", label="Type your message here:")
            submit_button = gr.Button("Send")

            history_state = gr.State([])

            submit_button.click(
                ask_question_gradio,
                inputs=[history_state, user_input],
                outputs=[chatbot, history_display],
                scroll_to_output=True
            )

            user_input.submit(
                ask_question_gradio,
                inputs=[history_state, user_input],
                outputs=[chatbot, history_display],
                scroll_to_output=True
            )

# Launch the Gradio interface
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ffde0003f396f72fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [39]:
# Start the interactive chat
print("Welcome to the RAG Chat Model! Ask any question (type 'exit' to quit):")
while True:
  print("\n\n")
  question = input("Your question: ")
  if question.lower() == 'exit':
    print("Exiting the chat.")
    break
  rag_chat_model.ask_question(question)

Welcome to the RAG Chat Model! Ask any question (type 'exit' to quit):



Your question: what is ideal gas law
No relevant documents found. Please try with a different query.



Your question: what is meaning of sick
No relevant documents found. Please try with a different query.





KeyboardInterrupt: Interrupted by user

In [38]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from typing import List

# Step 1: Load Model and Tokenizer
model_name = "unsloth/phi-3.5-mini-instruct-bnb-4bit"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    attn_implementation="flash_attention_2",  # Use flash attention for efficient inference
    trust_remote_code=True,
    torch_dtype=torch.float16  # Memory efficient
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Set PAD token if it's not set already
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Step 2: Load PEFT Model (assuming a LoRA fine-tuned model is available)
model = PeftModel.from_pretrained(model, "ayshwaryaninet1/lora_model19")

# Step 3: Set Device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Step 4: Setup Embedding Model and Vector Store
embedding_model_name = "BAAI/bge-small-en-v1.5"
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True}
)

# Initialize Chroma vector store
vectorstore = Chroma(
    persist_directory="/content/drive/MyDrive/RAG/vector1",
    embedding_function=embedding_model
)

# Create Retriever from Vector Store
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Retrieve the top 5 results

# Step 5: Define the RAGChatModel Class for Query Handling
class RAGChatModel:
    def __init__(self, retriever, model, tokenizer, device, max_token_limit=512):
        self.retriever = retriever
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.max_token_limit = max_token_limit

        # Prompt template with explicit instructions for accuracy
        self.template_standard = """
        You are an assistant specialized in chemical engineering. Please answer the following question based on the provided context. Make sure the answer is strictly based on the provided information.

        Context:
        {context}

        Question:
        {question}

        Answer:
        """

    def format_docs(self, docs: List, full_content=True, max_pages=3) -> str:
        """Format the documents to be used as context in the prompt, limiting to max_pages."""
        if full_content:
            formatted_docs = "\n\n".join(f"Page {doc.metadata.get('page', 'N/A')}: {doc.page_content}" for doc in docs[:max_pages])
        else:
            formatted_docs = "Information available in pages: " + ", ".join(f"{doc.metadata.get('page', 'N/A')}" for doc in docs[:max_pages])
        return formatted_docs

    def get_prompt(self, docs: List, question: str) -> str:
        """Generate the prompt based on retrieved documents."""
        context = self.format_docs(docs, full_content=True, max_pages=3)
        prompt = self.template_standard.format(context=context, question=question)
        return prompt

    def ask_question(self, question: str):
        """Main function to retrieve relevant context and generate a response."""
        # Retrieve relevant documents based on the question
        docs = self.retriever.invoke(question)
        if not docs:
            print("No relevant documents found. Please try with a different query.")
            return

        # Create the prompt from retrieved documents
        prompt = self.get_prompt(docs, question)

        # Tokenize the prompt and move input tensors to the correct device
        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True).to(self.device)

        # Generate the response with the model
        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=200,  # Generate up to 200 new tokens
                do_sample=True,
                temperature=0.3,  # Low temperature for more deterministic output
                top_p=0.9,        # Nucleus sampling for balanced diversity
                repetition_penalty=1.1  # Penalty for repeating phrases
            )

        # Decode the response and clean it
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = self.extract_clean_answer(response)
        print("Answer:", answer)

    def extract_clean_answer(self, raw_output: str) -> str:
        """Extract and clean the generated answer."""
        if "Answer:" in raw_output:
            return raw_output.split("Answer:")[-1].strip()
        return raw_output.strip()

# Step 6: Initialize and Run the RAGChatModel
rag_chat_model = RAGChatModel(
    retriever=retriever,
    model=model,
    tokenizer=tokenizer,
    device=device
)

# Step 7: Interactive Chat
print("Welcome to the Chemical Engineering RAG Chat Model! Ask any question (type 'exit' to quit):")
while True:
    print("\n")
    question = input("Your question: ")
    if question.lower() == 'exit':
        print("Exiting the chat.")
        break

    try:
        rag_chat_model.ask_question(question)
    except Exception as e:
        print(f"An error occurred: {e}")


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Welcome to the Chemical Engineering RAG Chat Model! Ask any question (type 'exit' to quit):


Your question: what is ideal gas law
No relevant documents found. Please try with a different query.




KeyboardInterrupt: Interrupted by user